In [15]:
import os
import shutil
import glob
import fiftyone as fo

In [ ]:
def create_new_dataset(original_ds_path, new_ds_path):
    """ 
    """
    os.mkdir(new_ds_path)
    os.mkdir(f'{new_ds_path}/images')
    os.mkdir(f'{new_ds_path}/images/train')
    os.mkdir(f'{new_ds_path}/images/val')
    os.mkdir(f'{new_ds_path}/labels')
    os.mkdir(f'{new_ds_path}/labels/train')
    os.mkdir(f'{new_ds_path}/labels/val')
    
    for filepath in glob.glob(f'{original_ds_path}/train/*.jpg'):
        shutil.copy2(filepath, f'{new_ds_path}/images/train')
    for filepath in glob.glob(f'{original_ds_path}/train/*.txt'):
        shutil.copy2(filepath, f'{new_ds_path}/labels/train')
    for filepath in glob.glob(f'{original_ds_path}/val/*.jpg'):
        shutil.copy2(filepath, f'{new_ds_path}/images/val')
    for filepath in glob.glob(f'{original_ds_path}/val/*.txt'):
        shutil.copy2(filepath, f'{new_ds_path}/labels/val')
        
    s = f'path: {new_ds_path} \n'
    s += 'train: ./images/train/ \n'
    s += 'val: ./images/val/ \n'
    s += 'names: \n'
    s += '  0: live \n'
    s += '  1: dead \n'
    s += '  2: vcut \n'
    with open(f'{new_ds_path}/dataset.yaml', 'w') as f:
        f.write(s)

In [ ]:
def yolo2fiftyone(name=fo_dataset_name, dataset_dir=dataset_dir, splits=["train", "val"]):
    """ 
    Imports a dataset in YOLO5 format into FiftyOne, using tags to mark the samples in each split 
    """ 
    dataset = fo.Dataset(name, persistent=True)
    for split in splits:
        dataset.add_dir(
            dataset_dir=dataset_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            split=split,
            tags=split,
    )
    return dataset

In [ ]:
def create_datetime_tag():
    """
    """
    pass

In [ ]:
def remove_autocorrelated_images():
    """ 
    """ 
    pass

import numpy as np
from numpy.linalg import norm

def cosine_similarity(a, b):
    return np.dot(a,b)/(norm(a)*norm(b))
 
# a = np.array([2,1,2,3,2,9])
# b = np.array([3,4,2,4,5,5])
# cosine_similarity(a, b)


sorted_by_datetime_view = dataset.load_saved_view('sorted_by_datatime_view')

thresh = 0.92

first_sample = True
for sample in sorted_by_datetime_view:
    if first_sample:
        current_embeddings = sample.embeddings
        similarity = 0.0
        first_sample = False
    else:
        previous_embeddings = current_embeddings
        current_embeddings = sample.embeddings
        similarity = cosine_similarity(previous_embeddings, current_embeddings)
        sample['similarity_with_prev_img'] = similarity
    if similarity > thresh:
        sample.tags.append(f'similarity>{thresh}')
    else:
        sample.tags.append('similarity OK') 
    sample.save()

In [ ]:
# MAIN
original_ds_path = '/home/aubrey/Desktop/Guam07-training-set/datasets/3class-no-symlinks'
new_ds_path = '/home/aubrey/datasets/Guam07v3'
fo_dataset_name = 'Guam07v3'

# wrangle dataset into YOLOv5 format
# create_new_dataset(original_ds_path, new_ds_path)

# Import dataset into FiftyOne
# dataset = yolo2fiftyone(name=name, dataset_dir=new_ds_path)

# Open dataset in FiftyOne app in browser
dataset = fo.load_dataset(fo_dataset_name)
fo.launch_app(dataset, auto=False)